In [66]:
import pathlib as pl
import pandas as pd
from asammdf import MDF
import ipywidgets as widgets

In [67]:
# E:\Messdaten\Setup-I\2021-04-28 - Gut 1 as pathlib path
kidaq_folder = r'E:\Messdaten\Setup-I\2021-04-28 - Gut 1\KiDAQ'

In [68]:
# create a path object
path = pl.Path(kidaq_folder)
# list all folders in the path
folders = [str(x) for x in path.iterdir() if x.is_dir()]
# create a widget to select the folders
tags = widgets.TagsInput(
    value=folders,
    allowed_tags=folders,
    allow_duplicates=False,
    description='Folders:',
)
display(tags)

TagsInput(value=['E:\\Messdaten\\Setup-I\\2021-04-28 - Gut 1\\KiDAQ\\1450rpm', 'E:\\Messdaten\\Setup-I\\2021-0…

In [69]:
# set window size
window_size_int = widgets.BoundedIntText(
    value=100000,
    min=0,
    max=1000000,
    step=1,
    description='Window Size:',
    disabled=False
)
display(window_size_int)

BoundedIntText(value=100000, description='Window Size:', max=1000000)

In [70]:
# set window size
step_size_int = widgets.BoundedIntText(
    value=100000,
    min=0,
    max=1000000,
    step=1,
    description='Step Size:',
    disabled=False
)
display(step_size_int)

BoundedIntText(value=100000, description='Step Size:', max=1000000)

In [71]:
agg_type_dropdown = widgets.Dropdown(
    options=['median', 'std', 'min', 'max'],
    value='median',
    description='Aggregation:',
    disabled=False,
)
display(agg_type_dropdown)

Dropdown(description='Aggregation:', options=('median', 'std', 'min', 'max'), value='median')

In [72]:
# read mdf file in loop and aggregate data
for folder in tags.value:
    # transform folder string to path object
    folder_path = pl.Path(folder)
    # get all mdf files in the folder
    mdf_files = [x for x in folder_path.iterdir() if x.suffix == '.mf4']
    # cut the folder path to get the folder name
    folder_name = folder_path.parts[-4:]
    # create a new folder in output/plots folder with todays date
    output_folder = pl.Path('output/plots') / pl.Path(f'{agg_type_dropdown.value}') / pl.Path(folder_name[0]) / pl.Path(folder_name[1]) / pl.Path(folder_name[2]) / pl.Path(folder_name[3])
    # create the folder if it exists replace it
    output_folder.mkdir(parents=True, exist_ok=True)
    # loop over all mdf files in the folder
    for mdf_file in mdf_files:
        # create a mdf object
        mdf_obj = MDF(mdf_file)
        # transform to dataframe
        df = mdf_obj.to_dataframe()
        # aggregate data
        df_agg = df[['p1', 'p2']].rolling(window=window_size_int.value, step=step_size_int.value).agg(agg_type_dropdown.value)
        # give title to plot 
        title = f'{agg_type_dropdown.value} of {mdf_file.stem}'
        # plot data as figure
        df_figure = df_agg.plot(title=title, grid=True).get_figure()
        # save figure and dont show it
        df_figure.savefig(output_folder / pl.Path(f'{mdf_file.stem}.png'), dpi=300)
        # close figure
        df_figure.clf()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>